In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

from math import exp

In [2]:
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 120)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", "{:,.2f}".format)

pio.templates.default = "plotly_white"
pio.kaleido.scope.default_scale = 2

gruvbox_colors = [
    "#458588",
    "#FABD2F",
    "#B8BB26",
    "#CC241D",
    "#B16286",
    "#8EC07C",
    "#FE8019",
]

In [3]:
def co2_exhalation(
    H_forCO2 = 1.8,
    W_forCO2 = 80,
    activity_type: int = 0,
) -> float:
    """Returns the exhalation rate for a person

    Args:
        H_forCO2 (float, optional): Height of the individual. Defaults to 1.8.
        W_forCO2 (int, optional): Weight of the individual. Defaults to 80.
        activity_type (int, optional): Scale of activity. Defaults to 0.

    Returns:
        float: exhalation rate for the person scaled to calculate ppm in the end
    """
    # Base value for CO2 emission (based on https:#doi.org/10.1111/ina.12383)
    AD_forCO2 = 0.202*(H_forCO2**0.725)*(W_forCO2**0.425); # DuBois surface area, m^2
    RQ_forCO2 = 0.85  # respiratory quotient (dimensionless)
    co2_exhRate_without_met = (0.00276 * AD_forCO2 * RQ_forCO2) / (
        0.23 * RQ_forCO2 + 0.77
    )  # ltr/s/met
    met_ref = 1.15  # reference metabolic rate, met

    # Metabolic rate applied to to co2_exhRate_ref (based on https:#doi.org/10.1111/ina.12383).
    # (i) sitting/breathing, (ii) standing/light exercise, (iii) heavy exercise
    # in the paper these are taken for:
    # (i) average from range in sitting quietly 1.15 met (see met_ref above)
    # (ii) standing quietly,  light exercise  1.3 met
    # (iii) calisthenics, moderate effort 3.8 met
    metabolic_rate_forCO2 = [
        met_ref,
        1.3,
        3.8,
    ]  # metabolic rate based on activity, met

    co2_exhRate = (
        co2_exhRate_without_met * metabolic_rate_forCO2[activity_type]
    )  # ... indicative CO2 emission rate, ltr/s

    co2_exhRate = co2_exhRate / 1000  # ... m3/s
    co2_exhRate = co2_exhRate * 10**6  # ...scale to calculate ppm in the end

    return co2_exhRate

In [4]:
def co2_concentration(
    co2_Rate,
    Ar: float = 100,
    Hr: float = 3,
    outside_air: int = 100,
    permanence: float = 120,
    ACH_custom: float = 20,
    s_ACH_type: int = 6,
) -> pd.DataFrame:
    """Calculates the conentration of CO2 in the room over time

    Args:
        c02_exhRate (float): Rate of exhalation of a person.
        Ar (int, optional): Area of the room. Defaults to 100.
        Hr (int, optional): Height of the room. Defaults to 3.
        outside_air (int, optional): [description]. Defaults to 100.
        permanence (int, optional): Time of permanence in the room in minutes. Defaults to 60.
        ACH_custom (int, optional): Custom value of ACH. Defaults to 1.
        s_ACH_type (int, optional): Selection of type of ACH. Defaults to 6 which means custom.

    Returns:
        pd.DataFrame: DataFrame with the columns of people over time and concentration of CO2
    """
    t0 = 0
    tMax = permanence * 60
    dt = tMax / 400  # ... time increment, s
    time_series = np.arange(start=t0, stop=tMax, step=dt).tolist()

    V = Ar * Hr  # ... room volume (m^3)

    result = pd.DataFrame(time_series, columns=["time"])

    # Define background CO2 (hope this does not change a lot...)
    co2_background = 415  # ... CO2 outdoors, ppm

    # Sets ACH based on the modes set at the interface
    sACH = [0.3, 1, 3, 5, 10, 20, 999]
    ACH = ACH_custom if s_ACH_type == 6 else sACH[s_ACH_type]
    ACH_fresh = (ACH * outside_air) / 100  # ...1/h
    vent_fresh = ACH_fresh / 3600  # ... 1/s
    loss_rate_co2 = vent_fresh

    co_2 = []

    for row in result.itertuples():
        if loss_rate_co2 > 0:
            co_2.append(
                co2_background
                + (
                    (co2_Rate) / V / loss_rate_co2
                    + (
                        (co2_background if row.Index == 0 else co_2[row.Index - 1])
                        - (co2_Rate) / V / loss_rate_co2
                        - co2_background
                    )
                    * exp(-loss_rate_co2 * dt)
                )
            )
        else:
            co_2.append(
                (co2_background if row.Index == 0 else co_2[row.Index - 1])
                + ((row.n_people * co2_Rate) / V) * dt
            )

    result["co_2"] = co_2

    return result

In [5]:
full_co2_exh = 20 *co2_exhalation(activity_type=2)

In [86]:
full_scale_co2 = co2_concentration(full_co2_exh, Ar=31.71, Hr=2.3, ACH_custom=50, permanence=60)
scaled_co2 = co2_concentration(0.04212*full_co2_exh, Ar=3.84, Hr=0.8, ACH_custom=50, permanence=60)

In [87]:
scaled_co2.head()

,time,co_2
0,0.00,457.67
1,9.00,495.32
2,18.00,528.55
3,27.00,557.87
4,36.00,583.75


In [88]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=full_scale_co2["time"], y=full_scale_co2["co_2"], name="Full Scale", line=dict(color=gruvbox_colors[0], width=2)))
fig.add_trace(go.Scatter(x=scaled_co2["time"], y=scaled_co2["co_2"], name="Scaled CO<sub>2</sub>", line=dict(color=gruvbox_colors[2], width=2)))

fig.update_layout(title="CO<sub>2</sub> concentrations for fullscale and model",
                   xaxis_title="Time",
                   yaxis_title="CO<sub>2</sub> (ppm)",
                   width=800,
    height=500,)

fig.show()

In [89]:
coef_wo_ach = 0.005265